In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self, input_size = 28*28, hidden_size = 128, output_size =62):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [3]:
pip install torch ray


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 12.8 MB/s eta 0:00:00


In [39]:
import ray

@ray.remote(num_cpus=1, num_gpus=1)  # Adjust resources as needed
class ClientActor:
    def __init__(self, model, loss_fn, data_loader):
        self.model = model
        self.model.load_state_dict(global_model.state_dict())
        self.optimizer = torch.optim.Adam(self.model.parameters())
        self.loss_fn = loss_fn
        self.data_loader = data_loader

    def train(self):
        self.model.train()
        data = torch.tensor(datasets[0]['images']).view(-1,28*28)
        #data = data.view(data.size(0), -1)
        print(data.shape)
        target = torch.tensor(self.data_loader['labels']).type(torch.LongTensor)
        self.optimizer.zero_grad()
        output = self.model(data)
        loss = self.loss_fn(output, target)
        loss.backward()
        self.optimizer.step()
        return self.model.state_dict()
    def set_model(self, global_model_state_dict):
        self.model.load_state_dict(global_model_state_dict)


In [5]:
from google.colab import drive
import numpy as np
drive.mount('/content/drive')
path_to_data = 'drive/MyDrive/Colab Notebooks/train_data.npy'
user_ids = [0, 1, 2, 3, 4]  # Replace with chosen user IDs
datasets = np.load(path_to_data, allow_pickle=True)

Mounted at /content/drive


In [40]:
#ray.init()  # Initialize Ray

# Assuming you have a DataLoader, loss function, and optimizer ready
# For example:
# data_loader = DataLoader(...)
loss_fn = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters())

# Create client actors
num_clients = 4  # Adjust as needed
#client_actors = [ClientActor.remote(SimpleNN(), loss_fn, datasets[i]) for i in range(num_clients)]
#client_actors = [ClientActor(SimpleNN(), loss_fn, datasets[i]) for i in range(num_clients)]



In [33]:
print(torch.tensor(datasets[0]['images']).view(-1,28*28).shape)

torch.Size([344, 784])


In [42]:
num_rounds = 10  # Define the number of communication rounds

for round in range(num_rounds):
    print(round)
    num_clients = 4  # Adjust as needed
    selected_clients = np.random.choice(np.arange(len(datasets)), num_clients, replace=False)
    client_actors = [ClientActor.remote(SimpleNN(), loss_fn, ray.put(datasets[i])) for i in selected_clients]
    # Sample clients and train in parallel
    #updated_models = [actor.train() for actor in client_actors]
    updated_models = ray.get([actor.train.remote() for actor in client_actors])

    # Aggregate model updates (simple averaging for demonstration)
    global_model_state_dict = {k: torch.mean(torch.stack([model[k] for model in updated_models]), dim=0) for k in updated_models[0]}

    # Update global model
    for actor in client_actors:
        actor.set_model.remote(global_model_state_dict)


0


2023-11-16 21:40:35,433	WARNING worker.py:2074 -- Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 2064, in ray._raylet.task_execution_handler
  File "python/ray/_raylet.pyx", line 1921, in ray._raylet.execute_task_with_cancellation_handler
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/function_manager.py", line 574, in load_actor_class
    actor_class = self._load_actor_class_from_gcs(
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/function_manager.py", line 669, in _load_actor_class_from_gcs
    class_name = ensure_str(class_name)
  File "/usr/local/lib/python3.10/dist-packages/ray/_private/utils.py", line 243, in ensure_str
    assert isinstance(s, bytes)
AssertionError
An unexpected internal error occurred while the worker was executing a task.
2023-11-16 21:40:35,443	WARNING worker.py:2074 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead

RayActorError: ignored